### Character Generator Tester
Just hit "Kernel > Restart and Run All", wait 20 seconds, then scroll to the bottom to create your own prompts and character descriptions!

---

In [1]:
# import libraries and files
from quick_gpt import QuickGPT
import quick_ner as NER
import blurb_gen as BlurGen
from ngrams_model import NGramGen
from holy_grail_traits import HolyGrailTraits
from mad_libs_traits import MadLibsTraits
import re
import random
import math
import numpy as np

# import the brains
print("+++ IMPORTING TRANSFORMER BRAIN +++")
char_brain = QuickGPT(model_type='gpt2-medium',corpus="character_corpuses/character_descriptions_spark_GENERIC.txt")

#astro_brain = NGramGen("character_corpuses/character_descriptions_astrology_CLEAN3.txt",4)

print("+++ IMPORTING HOLY GRAIL TRAITS +++")
hg_traits = HolyGrailTraits()

print("+++ IMPORTING MAD LIBS TRAITS +++")
ml_traits = MadLibsTraits()

+++ IMPORTING TRANSFORMER BRAIN +++
+++ IMPORTING HOLY GRAIL TRAITS +++
> imported sentence encoder model!


100%|██████████| 7/7 [00:01<00:00,  6.02it/s]


> imported and encoded traits for ['astro', 'adj', 'dark_mbti', 'hexaco', 'mbti', 'tt_flaws', 'wattpad']!
+++ IMPORTING MAD LIBS TRAITS +++
> imported sentence encoder model!
> importing and encoding: NOUN
> importing and encoding: PNOUN
> importing and encoding: ADJ
> importing and encoding: PADJ
> importing and encoding: VERB
> importing and encoding: ANOUN
> importing and encoding: EVERB


In [2]:
# generate a character description using the brain with the generic replacements (cuts off before a new desc is given)
def gengenCharDesc(txt,n=5):
    global char_brain
    d_set = char_brain.gen_txt(txt,{'max_new_tokens':70,"num_return_sequences":n,'top_k':4, "repetition_penalty":1})
    
    #parse the set
    new_d_set = [d.split("--")[0] for d in d_set]
    
    #replace the generic placeholders
    new_d2_set = [NER.replaceGeneric(d,txt) for d in new_d_set]
    
    #remove stragglers just in case
    new_d3_set = [re.sub("#(X|Y)\_?([0-9]+)?#?","",d) for d in new_d2_set]
    
    return new_d_set, new_d3_set

def genAstroDesc(txt,n=7,s=3,tr=(6,12)):
    # generate a set
    a_set = [astro_brain.generate() for i in range(n)]
    a_set2 = [[x] for x in a_set if len(x.split(" ")) in range(tr[0],tr[1])]
    
    #cluster by relavence to the original prompt (like the W2V bullet grouping)
    best_astro, removed = BlurGen.getBestBlurbs(txt,a_set2,math.ceil(s/2),math.floor(s/2))  #get the top S(+1?) blurbs (S/2 ceil closest, S/2 floor furthest)
    
    return best_astro, removed
    

In [ ]:
print("NOTE: Add '-- [NAME] --' at the end to get a specific character's description, or one will randomly be picked.\nEx. 'Alice fell down a rabbit hole. -- Alice --'\n********************************************\n")
MODE_TYPE = input("Which mode type would you like?\n - full (default) : generates a paragraph backstory/spoiler description\n - blurb : generates bullet point descriptions\n - grail : selects related traits from scraped websites (shown in distinct emoji bullets)\n - madlib : short templative character summaries\n\n")
print("********************************************\n")


# infinite loop
while True:
    user_prompt = input("Tell a story: ")
    
    # break loop
    if user_prompt == "exit":
        break

    # select a random character if not given one
    if "--" not in user_prompt:
        all_names = NER.getCharacters(user_prompt)
        if len(all_names) == 0:
            all_names = NER.getProperNouns(user_prompt)
        if len(all_names) == 0: 
            print("ERROR: No character names found... Try capitalizing them!")
            continue
    
        name_pick = f"-- {random.choice(all_names)} --"
        user_prompt += f"\n{name_pick}"
        print(name_pick)

    # generate descriptions
    max_prompts = 3 if MODE_TYPE == "grail" else 5
    _, descs = gengenCharDesc(user_prompt,max_prompts)
    
    #bullet point mode
    if MODE_TYPE == "blurb":
        bullets = []
        # add all blurbs
        for d in descs:
            bpt = BlurGen.bulletPointText(d)
            bullets += [(NER.descPostProc(user_prompt,x)) for x in bpt]  #replace any repeated instances of MC name

        blurb_groups = BlurGen.group_blurbs_W2V(bullets,0.4)        #cluster blurbs together
        best_blurbs, rem_blurb = BlurGen.getBestBlurbs(user_prompt,blurb_groups,4,3)  #get the top 7 blurbs (4 closest, 3 furthest)
            
#         # Create astrology descriptions (show them first)
#         ads, rem_ads = genAstroDesc(user_prompt,n=100,s=5)
#         for a in ads:
#             print(f"⭐️ {a}")
#         for a in rem_ads[:5]:
#             print(f"⭐️ {BlurGen.strike(a)}")
#         print("")
 
        # show regular blurbs extracted
        for b in best_blurbs:
            print(f"- {b}")
        for b in rem_blurb:
            print(f"- {BlurGen.strike(b)}")
            
    elif MODE_TYPE == "grail":
        best_traits, bad_traits = hg_traits.bestTraits(user_prompt,descs,3)
        
        for bet in best_traits:
            print(bet)
        print("")
        for bat in bad_traits:
            print(BlurGen.strike(bat))
            
    elif MODE_TYPE == "madlib":
        best_traits, bad_traits = ml_traits.bestTraits(user_prompt,descs,3)
        
        for bet in best_traits:
            print(f"- {bet}")
        print("")
        for bat in bad_traits:
            print(f"- {BlurGen.strike(bat)}")
            
    
    # full bio mode
    else:
        for i in range(len(descs)):
            print(f"> {descs[i]}\n")
            print()
    print("\n------------------------------------------------------------\n")

NOTE: Add '-- [NAME] --' at the end to get a specific character's description, or one will randomly be picked.
Ex. 'Alice fell down a rabbit hole. -- Alice --'
********************************************

Which mode type would you like?
 - full (default) : generates a paragraph backstory/spoiler description
 - blurb : generates bullet point descriptions
 - grail : selects related traits from scraped websites (shown in distinct emoji bullets)
 - madlib : short templative character summaries

madlib
********************************************

Tell a story: Far beyond the celestial star X-39742 lurked the spaceship Abenteuer. Nobody had heard from the ship in decades. So in the year 3422, when the communication arrays began blinking with urgent mayday signals from the craft, Captain Victor was perplexed. Perplexed, and unnerved.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-- Victor --
- a supercilious astronomer
- a frank astronaut naming professors
- a conscientious sailor

- a̶ ̶p̶a̶t̶e̶r̶n̶a̶l̶ ̶f̶a̶m̶i̶l̶i̶a̶l̶ ̶n̶o̶v̶e̶l̶i̶s̶t̶
- a̶ ̶m̶e̶d̶d̶l̶e̶s̶o̶m̶e̶ ̶a̶u̶t̶h̶o̶r̶ ̶b̶o̶b̶i̶n̶g̶ ̶u̶n̶c̶l̶e̶s̶
- a̶n̶ ̶e̶a̶s̶y̶-̶g̶o̶i̶n̶g̶ ̶j̶e̶s̶t̶e̶r̶ ̶w̶h̶o̶ ̶s̶p̶o̶i̶l̶s̶ ̶c̶o̶u̶s̶i̶n̶s̶
- a̶ ̶m̶e̶d̶d̶l̶e̶s̶o̶m̶e̶ ̶c̶a̶b̶ ̶d̶r̶i̶v̶e̶r̶ ̶w̶h̶o̶ ̶v̶e̶n̶t̶u̶r̶e̶s̶ ̶s̶i̶s̶t̶e̶r̶s̶
- a̶ ̶s̶t̶o̶i̶c̶ ̶m̶e̶e̶k̶ ̶a̶u̶t̶h̶o̶r̶
- a̶ ̶c̶o̶u̶r̶a̶g̶e̶o̶u̶s̶ ̶g̶a̶l̶l̶a̶n̶t̶ ̶a̶u̶t̶h̶o̶r̶ ̶a̶n̶d̶ ̶e̶n̶i̶g̶m̶a̶t̶i̶c̶ ̶l̶i̶b̶r̶e̶t̶t̶i̶s̶t̶
- a̶ ̶c̶o̶m̶p̶o̶s̶e̶d̶ ̶n̶o̶v̶e̶l̶i̶s̶t̶ ̶s̶p̶o̶i̶l̶i̶n̶g̶ ̶o̶w̶n̶e̶r̶s̶
- a̶ ̶s̶h̶o̶r̶t̶-̶t̶e̶m̶p̶e̶r̶e̶d̶ ̶r̶e̶v̶e̶r̶e̶n̶t̶i̶a̶l̶ ̶w̶r̶i̶t̶e̶r̶
- a̶n̶ ̶u̶n̶c̶e̶r̶t̶a̶i̶n̶ ̶m̶u̶r̶a̶l̶i̶s̶t̶ ̶w̶h̶o̶ ̶a̶p̶p̶e̶a̶r̶s̶ ̶a̶p̶p̶e̶a̶r̶a̶n̶c̶e̶s̶
- a̶n̶ ̶u̶n̶c̶e̶r̶t̶a̶i̶n̶ ̶g̶a̶l̶l̶a̶n̶t̶ ̶a̶r̶t̶i̶s̶t̶
- a̶ ̶c̶o̶u̶r̶a̶g̶e̶o̶u̶s̶ ̶a̶c̶t̶o̶r̶ ̶r̶e̶i̶g̶n̶i̶n̶g̶ ̶s̶t̶y̶l̶e̶s̶
- a̶ ̶f̶r̶a̶n̶k̶ ̶a̶c̶t̶o̶r̶ ̶w̶h̶o̶ ̶c̶o̶m̶b̶s̶ ̶s̶i̶n̶g̶e̶r̶s̶
- a̶ ̶s̶t̶o̶i̶c

---

**Milk's copypasta**: 

Alice fell down a rabbit hole. Which was not surprising - as she had been heavily drinking at her friend Hatter's bar: the Spilled Teapot. But tonight was a special occasion. She had just graduated with her degree in philosophy.

Nat and Ash slept in a crashed space shuttle on the Moon. Ash did contracting work as an engineer fixing up the locals' gadgets, while Nat was a hoverboard pizza delivery robot.

Deep in the forests of Abaluria, a troupe of bandits waited to ambush Maron, the high prince of Denetir. The leader of the bandits was a fierce woman. She was named Peka after the local plant that grew rough and wild and dragged men beneath the surface of the earth. She was doing a great job of living up to her name.

Far beyond the celestial star X-39742 lurked the spaceship *Abenteuer*. Nobody had heard from the ship in decades. So in the year 3422, when the communication arrays began blinking with urgent mayday signals from the craft, Captain Victor was perplexed. Perplexed, and unnerved.

Jenny sat eagerly by the phone. After their date at the *Ristorante Calafanti* Marco had told her he would call at 7pm on the dot. Well, now it was 6.58pm and her whole body was tense, fingers hovering over the receiver. The minutes ticked past. 6.59pm. 7pm. 7.01pm. When the phone finally rang, Jenny snatched it up “Hello… oh… hi, Grandma.”

Niamh had always lived in Cashel. She knew the seasons like the back of her hand. She knew the name of each sheep and the bark of each dog. At 84 years of age, she was still sharp as a needle, and nothing much got past her narrowed eyes. Which was why, when Daithi O’Shea walked down Main Street in the middle of the night, Niamh knew exactly where he had been. And what he had been up to.

Tammy knew she wasn’t allowed to visit the abandoned house at the edge of the village of Centaury. She heard the adults whispering about it at the back of church on Sundays. Willa’s mother said it was full of junkies, “and… worse”, and Father Jeremy nodded along. But Tammy wouldn’t be swayed. Centaury was safe, after all, wasn’t it?

Detective Sergeant Fanshaw surveyed the evidence with a grim expression on her face. Blood, check. Weapon, check. Victims? Check, check, check, and check. The whole Savoy family lay there at her feet: Jimmy, Sara, Jimmy Junior and Betty. A quadruple homicide in Gewgaw Manor. And it was only Monday.